In [102]:
import pandas as pd
import numpy as np

In [103]:
data = pd.read_csv(r"C:\Users\dhruv\OneDrive\Documents\Python Projects\Deep_Learning\Churn Prediction Model\Bank Customer Churn Prediction.csv")

In [104]:
data.head()

,customer_id,credit_score,country,gender,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn
0,15634602,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,15647311,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,15619304,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,15701354,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,15737888,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [105]:
data.drop(["customer_id"], axis = 1, inplace=True)

In [106]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder

In [107]:
scaler = StandardScaler()
one_encoder = OneHotEncoder(sparse_output=False)
lab_encoder = LabelEncoder()

In [108]:
one_encoder = OneHotEncoder(sparse_output=False)
temp = one_encoder.fit_transform(data[["gender"]])

one_hot_df = pd.DataFrame(temp, columns=one_encoder.get_feature_names_out(["gender"]))

data = pd.concat([data.drop(["gender"], axis = 1), one_hot_df], axis= 1 )


In [109]:
data.head()

,credit_score,country,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,gender_Female,gender_Male
0,619,France,42,2,0.00,1,1,1,101348.88,1,1.0,0.0
1,608,Spain,41,1,83807.86,1,0,1,112542.58,0,1.0,0.0
2,502,France,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0
3,699,France,39,1,0.00,2,0,0,93826.63,0,1.0,0.0
4,850,Spain,43,2,125510.82,1,1,1,79084.10,0,1.0,0.0


In [110]:
trans_balance = scaler.fit_transform(data[["balance"]])
data["balance"] = trans_balance

scaler2 = StandardScaler()
data["estimated_salary"] = scaler2.fit_transform(data[["estimated_salary"]])

scaler3 = StandardScaler()
data["credit_score"] = scaler2.fit_transform(data[["credit_score"]])

In [111]:
data.head()

,credit_score,country,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,gender_Female,gender_Male
0,-0.326221,France,42,2,-1.225848,1,1,1,0.021886,1,1.0,0.0
1,-0.440036,Spain,41,1,0.117350,1,0,1,0.216534,0,1.0,0.0
2,-1.536794,France,42,8,1.333053,3,1,0,0.240687,1,1.0,0.0
3,0.501521,France,39,1,-1.225848,2,0,0,-0.108918,0,1.0,0.0
4,2.063884,Spain,43,2,0.785728,1,1,1,-0.365276,0,1.0,0.0


In [112]:
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

country_encoder = OneHotEncoder(sparse_output=False, drop=None)

encoded_df = country_encoder.fit_transform(data[["country"]])

encoded_df = pd.DataFrame(encoded_df, columns=country_encoder.get_feature_names_out(["country"]))

encoded_df.index = data.index

data = pd.concat([data.drop(["country"], axis=1), encoded_df], axis=1)


In [113]:
data.head()

,credit_score,age,tenure,balance,products_number,credit_card,active_member,estimated_salary,churn,gender_Female,gender_Male,country_France,country_Germany,country_Spain
0,-0.326221,42,2,-1.225848,1,1,1,0.021886,1,1.0,0.0,1.0,0.0,0.0
1,-0.440036,41,1,0.117350,1,0,1,0.216534,0,1.0,0.0,0.0,0.0,1.0
2,-1.536794,42,8,1.333053,3,1,0,0.240687,1,1.0,0.0,1.0,0.0,0.0
3,0.501521,39,1,-1.225848,2,0,0,-0.108918,0,1.0,0.0,1.0,0.0,0.0
4,2.063884,43,2,0.785728,1,1,1,-0.365276,0,1.0,0.0,0.0,0.0,1.0


In [114]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard


In [115]:
from sklearn.model_selection import train_test_split
X = data.drop(["churn"], axis = 1)
y = data["churn"]

X_train, X_test , y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=42)

In [116]:
model = Sequential()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



In [117]:
import datetime
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tf_callback = TensorBoard(log_dir= log_dir, histogram_freq= 1, write_graph=False)
es = EarlyStopping(monitor='val_loss', patience= 5, restore_best_weights=True)

In [118]:
model.add(Dense(128, activation="relu", input_shape=(X_train.shape[1],)))

# Hidden layers
model.add(Dense(128, activation="relu"))
model.add(Dense(128, activation="relu"))

# Output layer for binary classification (1 unit with sigmoid activation)
model.add(Dense(1, activation="sigmoid"))


history = model.fit(
        X_train, y_train, validation_data = (X_test, y_test),
        epochs = 25,
        callbacks = [tf_callback, es]
)

Epoch 1/25


c:\Users\dhruv\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


219/219 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.7640 - loss: 0.5670 - val_accuracy: 0.8080 - val_loss: 0.4471
Epoch 2/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8019 - loss: 0.4556 - val_accuracy: 0.7993 - val_loss: 0.4547
Epoch 3/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7980 - loss: 0.4563 - val_accuracy: 0.8170 - val_loss: 0.4300
Epoch 4/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8069 - loss: 0.4465 - val_accuracy: 0.8210 - val_loss: 0.4248
Epoch 5/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7993 - loss: 0.4509 - val_accuracy: 0.8170 - val_loss: 0.4200
Epoch 6/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.7990 - loss: 0.4462 - val_accuracy: 0.8203 - val_loss: 0.4184
Epoch 7/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8088 - loss: 0.4359 - val_accuracy: 0.8160 - val_loss: 0.4225
Epoch 8/25
219/219 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8096 - loss: 0.4366 - val_accuracy: 0.8147 - val_

In [119]:
%load_ext tensorboard
%tensorboard --logdir "c:/Users/dhruv/OneDrive/Documents/Python Projects/Deep_Learning/Churn Prediction Model/logs/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6008 (pid 9560), started 0:00:27 ago. (Use '!kill 9560' to kill it.)